In [ ]:
import numpy as np
import pandas as pd

from polara import get_movielens_data
from polara.lib.earlystopping import early_stopping_callback

from scipy.sparse import diags, coo_matrix

from source.dataprep.dataprep import split_data_global_timepoint, generate_interactions_matrix
from source.evaluation.evaluation import topn_recommendations, model_evaluate, downvote_seen_items


from tqdm.notebook import tqdm

In [ ]:
data = get_movielens_data(include_time=True)
data_description = {
    'users':'userid',
    'items':'movieid',
    'feedback':'rating',
    'timestamp':'timestamp'
}

In [ ]:
training, testset_valid, holdout_valid, testset, holdout, data_index, data_description = split_data_global_timepoint(data=data, data_description=data_description)

In [ ]:
data_description = {
    'n_users':training.nunique()['userid'],
    'n_items':training.nunique()['movieid'],
    'users':'userid',
    'items':'movieid',
    'feedback':'rating',
    'timestamp':'timestamp'
}

# FPMC

In [ ]:
def generate_histories(data, data_description):
    histories = ...
    return histories

$$
x_{uij} = v^{U,I}_u \cdot v^{I,U}_j + v^{I,L}_j \cdot v^{L,I}_i
$$

In [ ]:
class FPMC:
    def __init__(self, model_config) -> None:
        self.model_config = model_config 
        # generator for interaction sampling
        self.rng = np.random.default_rng(seed=model_config['seed'])
        self.n_iters = model_config.get('epoch_iterations', 100000)
        self.n_users = model_config['n_users']
        self.n_items = model_config['n_items']
        # random initialization of model's factors
        self.sigma = model_config.get('sigma', 0.01)
        self.V_LI = self.rng.normal(loc=0.0, scale=self.sigma, size=(model_config['n_items'], model_config['dim']))
        self.V_IL = self.rng.normal(loc=0.0, scale=self.sigma, size=(model_config['n_items'], model_config['dim']))
        self.V_IU = self.rng.normal(loc=0.0, scale=self.sigma, size=(model_config['n_items'], model_config['dim']))
        self.V_UI = self.rng.normal(loc=0.0, scale=self.sigma, size=(model_config['n_users'], model_config['dim']))
        
    
    def sigmoid(self, x, cutoff = 10.0):
        sigmoid = ...

        
    def fit_partial(self, interactions, data_description, n_epochs=1, n_iters=None):
        # fit the model. repeated calls to this method will
        # cause training to resume from the current model state
        histories = generate_histories(interactions, data_description)
        iterations = n_iters if n_iters else len(interactions)
        for epoch in range(n_epochs):
            for _ in tqdm(range(iterations), desc='Training'):
                # sample the triplet user, positive, negative
                user = ...
                pos_index = ...
                pos = ...
                neg = ...
                
                if pos_index == 0:
                    # the first item in interaction history has no predecessor,
                    # so we should calculate only MF part
                    x_upos = ...
                    x_uneg = ...
                else:
                    prev = histories[user][pos_index - 1]
                    x_upos = ...
                    x_uneg = ...
                    
                delta = 1.0 - self.sigmoid(x_upos - x_uneg)
                
                # update MF part
                self.V_UI[user, :] += self.model_config['alpha'] * (
                    delta * (self.V_IU[pos, :] - self.V_IU[neg, :])
                    - self.model_config['L_UI'] * self.V_UI[user, :]
                    )
                self.V_IU[pos, :] += self.model_config['alpha'] * (
                    delta * self.V_UI[user, :] - self.model_config['L_IU'] * self.V_IU[pos, :]
                    )
                self.V_IU[neg, :] += self.model_config['alpha'] * (
                    -delta * self.V_UI[user, :] - self.model_config['L_IU'] * self.V_IU[neg, :]
                    )
                
                if pos_index > 0:
                    # if user's interaction is not the first interaction,
                    # update MC part
                    prev = histories[user][pos_index - 1]
                    self.V_IL[pos, :] += self.model_config['alpha'] * (
                        delta * self.V_LI[prev, :] - self.model_config['L_IL'] * self.V_IL[pos, :]
                        )
                    self.V_IL[neg, :] += self.model_config['alpha'] * (
                        -delta * self.V_LI[prev, :] - self.model_config['L_IL'] * self.V_IL[neg, :]
                        )
                    self.V_LI[prev, :] += self.model_config['alpha'] * (
                        delta * (self.V_IL[pos, :] - self.V_IL[neg, :])
                        - self.model_config['L_LI'] * self.V_LI[prev, :]
                        )
                        
    def folding_in(self, interactions, data_description, n_epochs=1, n_iters=None):
        # this function allows to do folding-in of new users.
        # it does the "half-step" of gd to update user embeddings
        # without updating the item factors in MF and MC parts
        histories = generate_histories(interactions, data_description)

        # last interaction of each user to generate scores
        last_interactions = np.array(
            [histories[u][-1] for u in range(len(histories))]
            )
        iterations = n_iters if n_iters else self.n_iters
        
        ...
        
        for epoch in range(n_epochs):
            for _ in tqdm(range(iterations), desc='Folding-in'):
                # sample the triplet user, positive, negative
                user = ...
                pos_index = ...
                pos = ...
                neg = ...

                if pos_index == 0:
                    x_upos = ...
                    x_uneg = ...
                else:
                    prev = histories[user][pos_index - 1]
                    x_upos = ...
                    x_uneg = ...
                
                delta = 1.0 - self.sigmoid(x_upos - x_uneg)
                
                # update only MF user embeddings
                V_UI_warmstart[user, :] += self.model_config['alpha'] * (delta * (self.V_IU[pos, :] - self.V_IU[neg, :]) - self.model_config['L_UI'] * V_UI_warmstart[user, :])
                
        scores = ...
        return scores

        

In [ ]:
fpmc_config = {
    'n_items':data_description['n_items'],
    'n_users':data_description['n_users'],
    'dim':128,
    'seed':2024,
    'alpha':0.05,
    'L_UI':0.01,
    'L_IU':0.01,
    'L_IL':0.01,
    'L_LI':0.01,
    
    'max_epochs':5
}

fpmc_model_test = FPMC(fpmc_config)

In [ ]:
fpmc_model_test.fit_partial(training, data_description, n_epochs=1, n_iters=10000)
scores_val = fpmc_model_test.folding_in(testset_valid, data_description, n_epochs=1, n_iters=10000)

In [ ]:
scores_val.shape

In [ ]:
scores_val

In [ ]:
fpmc_config = {
    'n_items':data_description['n_items'],
    'n_users':data_description['n_users'],
    'dim':128,
    'seed':2024,
    'alpha':0.05,
    'L_UI':0.01,
    'L_IU':0.01,
    'L_IL':0.01,
    'L_LI':0.01,
    
    'max_epochs':5
}

fpmc_model = FPMC(fpmc_config)

In [ ]:
factor_norms = {
    'UI':[],
    'IU':[],
    'LI':[],
    'IL':[]
    }

In [ ]:
n_epochs = 5

for epoch in tqdm(range(n_epochs)):
    fpmc_model.fit_partial(training, data_description, n_epochs=1, n_iters=100000)
    scores_val = fpmc_model.folding_in(testset_valid, data_description, n_epochs=1, n_iters=100000)
    downvote_seen_items(scores_val, testset_valid, data_description)
    recs = topn_recommendations(scores_val)
    print(f'epoch {epoch + 1}: {model_evaluate(recs, holdout_valid, data_description)}')

    # save the factor norms for future analysis
    factor_norms['UI'].append(np.linalg.norm(fpmc_model.V_UI, 'fro'))
    factor_norms['IU'].append(np.linalg.norm(fpmc_model.V_IU, 'fro'))
    factor_norms['IL'].append(np.linalg.norm(fpmc_model.V_IL, 'fro'))
    factor_norms['LI'].append(np.linalg.norm(fpmc_model.V_LI, 'fro'))

In [ ]:
scores = fpmc_model.folding_in(testset, data_description, n_epochs=1, n_iters=100000)
downvote_seen_items(scores, testset, data_description)
recs = topn_recommendations(scores)
print(model_evaluate(recs, holdout, data_description))

In [ ]:
import matplotlib.pyplot as plt

for factor in factor_norms.keys():
    plt.plot(factor_norms[factor], label=factor)
    
plt.ylabel('Frobenius norm')
plt.xlabel('Epoch')
plt.legend()
plt.show()

## Early stopping

In [ ]:
def build_model(Model_class, model_config, data, data_description, early_stop_config=None, iterator=None):
    # the model
    model = Model_class(model_config)
    if iterator is None:
        iterator = lambda x: x
    # early stoppping configuration
    es_config = check_early_stop_config(early_stop_config)
    # training
    for epoch in iterator(range(model_config['max_epochs'])):
        try:
            train_epoch(epoch, model, data, data_description, es_config)
        except StopIteration:
            break
    return model


def check_early_stop_config(early_stop_config):
    if early_stop_config is None:
        early_stop_config = {}
    try:
        es_dict = dict(
            early_stopper = early_stop_config['evaluation_callback'],
            callback_interval = early_stop_config['callback_interval'],
            warm_data = early_stop_config['warm_data'],
            holdout = early_stop_config['holdout'],
            stop_early = True
        )
    except KeyError:
        es_dict = dict(stop_early = False)
    return es_dict


def train_epoch(
    epoch, model, train, data_description, es_config,
):
    model.fit_partial(
        train,
        data_description,
        n_epochs=1,
        n_iters=50000,
    )
    if es_config['stop_early'] and ((epoch+1) % es_config['callback_interval'] == 0):
        # evaluate model and raise StopIteration if early stopping condition is met
        es_config['early_stopper'](epoch, model, es_config['warm_data'], es_config['holdout'], data_description)

In [ ]:
def warm_evaluator(model, warm_data, holdout, data_description, target_metric='hr', n_epochs=1, n_iters=50000):
    scores = ...
    downvote_seen_items(scores, warm_data, data_description)
    recs = topn_recommendations(scores)
    metrics = model_evaluate(recs, holdout, data_description)
    return metrics[target_metric]

In [ ]:
try_early_stop = early_stopping_callback(
        warm_evaluator, max_fails=1, verbose=True
)

early_stop_config = dict(
    evaluation_callback = try_early_stop,
    callback_interval = 1, # break between consequent evaluation in epochs
    holdout = holdout_valid,
    warm_data = testset_valid
)

In [ ]:
fpmc_params = build_model(
    FPMC,
    fpmc_config,
    training,
    data_description,
    early_stop_config=early_stop_config,
    iterator=tqdm
)

In [ ]:
print(f'Epochs: {try_early_stop.iter}, target metric: {try_early_stop.target}') 

# FOSSIL

$$
p_u(j|S^u_{t-1}, S^u_{t-2}, S^u_{t-L})\sim \beta_j +  \left[ \frac{1}{|I_u^+ \backslash \{j\}|} \sum_{j' \in I_u^+ \backslash \{j\}} \boldsymbol{P}_{j'}  + \sum_{k=1}^L (\eta_k+\eta_k^u)\boldsymbol{P}_{S_{t-k}^u}  , \boldsymbol{Q}_j\right]

$$

In [ ]:
class FOSSIL:
    def __init__(self, model_config) -> None:
        self.model_config = model_config
        # generator for interaction sampling
        self.rng = np.random.default_rng(seed=model_config['seed'])
        # random initialization of model's factors
        self.sigma = model_config.get('sigma', 0.01)
        self.n_users = model_config['n_users']
        self.n_items = model_config['n_items']
        self.alpha = model_config['alpha']
        self.lr = model_config['lr']
        self.L = model_config['markov_order']
        self.reg = model_config['regularization']
        self.dim = model_config['dim']
        
        
        self.P = self.rng.normal(loc=0.0, scale=self.sigma, size=(model_config['n_items'], model_config['dim']))
        self.Q = self.rng.normal(loc=0.0, scale=self.sigma, size=(model_config['n_items'], model_config['dim']))
        self.eta = self.rng.normal(loc=0.0, scale=self.sigma, size=(model_config['n_users'], model_config['markov_order']))
        self.eta_bias = np.zeros(self.L)
        self.bias = np.zeros(self.n_items)
        
        self.n_iters = model_config.get('epoch_iterations', 100000)

    def sigmoid(self, x, cutoff = 10.0):
        x_cutoff = max(min(cutoff, x), -cutoff)
        return 1.0 / (1.0 + np.exp(-x_cutoff))

    def compute_score(self, user_id, curr_hist, item):
        length = min(self.L, len(curr_hist))
        
        long_term = ...

        short_term = ...

        return ...
        
    def fit_partial(self, interactions, data_description, n_epochs, n_iters=None):
        # fit the model. repeated calls to this method will
        # cause training to resume from the current model state
        
        histories = generate_histories(interactions, data_description)
        iterations = n_iters if n_iters is not None else self.n_iters
        
        for epoch in range(n_epochs):
            for _ in tqdm(range(iterations), desc='Training'):
                
                user = self.rng.integers(0, self.n_users)
                user_history = histories[user]
                pos_index = self.rng.integers(1, len(histories[user]))
                curr_hist = user_history[:pos_index + 1]
                neg = self.rng.choice(np.setdiff1d(np.arange(self.model_config['n_items']), user_history))

                pos = curr_hist[-1]
                curr_hist = curr_hist[:-1]
                length = min(self.L, len(curr_hist))

                long_term = ...
                short_term = ...

                x_pos = ...
                x_neg = ...
                
                delta = 1.0 - self.sigmoid(x_pos - x_neg)
                
                # Gradients
                V_upd = self.lr * ( delta * np.power(len(curr_hist), -self.alpha) * (self.Q[pos, :] - self.Q[neg, :]) - self.reg * self.P[curr_hist, :])
                V_upd2 = self.lr * delta *  np.outer((self.eta_bias + self.eta[user, :])[:length], self.Q[pos, :] - self.Q[neg, :])
                Q_pos_upd = self.lr * ( delta * (long_term + short_term) - self.reg * self.Q[pos, :])
                Q_neg_upd = self.lr * ( -delta * (long_term + short_term) - self.reg * self.Q[neg, :])
                bias_pos_upd = self.lr * (delta - self.reg * self.bias[pos])
                bias_neg_upd = self.lr * (- delta - self.reg * self.bias[neg])
                eta_bias_upd = self.lr * (delta * np.dot(self.P[curr_hist[:-length-1:-1], :], self.Q[pos, :] - self.Q[neg, :]) - self.reg * self.eta_bias[:length])
                eta_upd = self.lr * (delta * np.dot(self.P[curr_hist[:-length-1:-1], :], self.Q[pos, :] - self.Q[neg, :]) - self.reg * self.eta[user, :length])

                # Update
                self.P[curr_hist, :] += V_upd
                self.P[curr_hist[:-length-1:-1], :] += V_upd2
                self.Q[pos, :] += Q_pos_upd
                self.Q[neg, :] += Q_neg_upd
                self.bias[pos] += bias_pos_upd
                self.bias[neg] += bias_neg_upd
                self.eta_bias[:length] += eta_bias_upd
                self.eta[user, :length] += eta_upd
                
                        
    def folding_in(self, interactions, data_description, n_epochs=1, n_iters = None):
        # this function allows to do folding-in of new users.
        # it does the "half-step" of gd to update user embeddings
        # without updating the item factors in MF and MC parts
        iterations = n_iters if n_iters is not None else self.n_iters
        
        histories = generate_histories(interactions, data_description)
        eta_warmstart = self.rng.normal(
            loc=0.0,
            scale=self.sigma,
            size=(len(histories),
            self.model_config['markov_order']))
        
        for epoch in range(n_epochs):
            for _ in tqdm(range(iterations), desc='Folding-in'):
                
                user = self.rng.integers(0, len(histories))
                user_history = histories[user]
                pos_index = self.rng.integers(1, len(histories[user]))
                curr_hist = user_history[:pos_index + 1]
                neg = self.rng.choice(
                    np.setdiff1d(
                        np.arange(self.model_config['n_items']),
                        user_history))

                pos = curr_hist[-1]
                curr_hist = curr_hist[:-1]
                length = min(self.L, len(curr_hist))

                    
                # Compute error
                x_pos = ...
                x_neg = ...
                delta = self.sigmoid(x_pos - x_neg) # sigmoid of the error
                
                # Compute Update
                eta_upd = self.lr * (delta * np.dot(self.P[curr_hist[:-length-1:-1], :], self.Q[pos, :] - self.Q[neg, :]) - self.reg * self.eta[user, :length])
                eta_warmstart[user, :length] += eta_upd
                
        
        scores = ...
        
        for user in range(len(histories)):
            curr_hist = histories[user]
            length = min(self.L, len(curr_hist))
            
            long_term = ...

            short_term = ...
            
            scores = ...

        return scores

        

In [ ]:
fossil_config = {
    'n_items':data_description['n_items'],
    'n_users':data_description['n_users'],
    'seed':2024,
    'dim':30,
    'alpha':0.2,
    'lr':0.01,
    'markov_order':2,
    'regularization':0.1,
}

fossil_model_test = FOSSIL(fossil_config)

In [ ]:
fossil_model_test.fit_partial(training, data_description, n_epochs=1, n_iters=10000)
scores_val = fossil_model_test.folding_in(testset_valid, data_description, n_epochs=1, n_iters=10000)

In [ ]:
fossil_config = {
    'n_items':data_description['n_items'],
    'n_users':data_description['n_users'],
    'seed':2024,
    'dim':30,
    'alpha':0.2,
    'lr':0.01,
    'markov_order':2,
    'regularization':0.1,
}

fossil_model = FOSSIL(fossil_config)

In [ ]:
n_epochs = 5

for epoch in tqdm(range(n_epochs)):
    fossil_model.fit_partial(training, data_description, n_epochs=1, n_iters=300000)
    scores_val = fossil_model.folding_in(testset_valid, data_description, n_epochs=1, n_iters=100000)
    downvote_seen_items(scores_val, testset_valid, data_description)
    recs = topn_recommendations(scores_val)
    print(f'epoch {epoch + 1}: {model_evaluate(recs, holdout_valid, data_description)}')


In [ ]:
scores_test = fossil_model.folding_in(testset, data_description, n_epochs=1, n_iters=100000)
downvote_seen_items(scores_test, testset, data_description)
recs = topn_recommendations(scores_test)
print(model_evaluate(recs, holdout, data_description))

In [ ]:
train_history_lengths = training.groupby(data_description['users']).size().values
fossil_user_factor = np.linalg.norm(fossil_model.eta, axis=1)

In [ ]:
plt.scatter(train_history_lengths, fossil_user_factor)
plt.ylabel('Norm of user vector')
plt.xlabel('Length of user history')
plt.show()

# SeqMF

$$
    \mathcal{L}(\bold{P}, \bold{Q}) = \frac{1}{2}\sum_{u}\|\bold{r}(\bold{Q}, \bold{p}_u)-\bold{a}_u\|^2_{\bold{C}_u} + \frac{\lambda}{2}\left( \sum_{u}\|\bold{p}_u\|_2^2+\|\bold{Q}\|_F^2 \right) \rightarrow \min_{\{\bold{Q}, \bold{p}_u\}}
$$

The loss is almost identical to the regular MF. The sequential setting of the model is hidden at the inference step:

\begin{equation}
    \bold{r}(\bold{Q}, \bold{p}_u) = \bold{Q}\bold{p}_u + \text{diag}(\bold{S}_u\bold{Q}\bold{Q}^\top)
\end{equation}

where $S_u$ is the item transition matrix of user $u$. It encodes how likely the transition from item $j$ to item $i$ to occur (recall sequential rules):

$$
    (\bold{S}_u)_{ij} = \frac{\sum_{k=2}^{|\mathcal{H}_u^{(t)}|} \bold{1}[j \rightarrow i] \bold{1}[i_{k-1}^u=j] \bold{1}[i_k^u=i]}{\sum_{k=2}^{|\mathcal{H}_u^{(t)}|} \bold{1}[i_{k-1}^u=i]}
$$

$\bold{C}_u$ is a diagonal matrix, containing weights based on item popularity ($N$ is number of items):
$$
    (\bold{C}_u)_{i} = \frac{d_{ui}^\gamma + \alpha}{\sum_{j}d_{ui}^\gamma + \alpha N}
$$

To speed up model's inference, the equation (1) can be simplified:

$$
\bold{r}(\bold{Q}, \bold{p}_u) = \bold{Q}\bold{p}_u + \text{diag}(\bold{S}_u\bold{Q}\bold{Q}^\top) \approx \bold{Q}\bold{p}_u + \bold{Q}\sum_{i\in s_u}\bold{q}_i = \bold{Q}\bold{p}_u + h_u(\bold{Q})
$$

where the summation in second term goes over the last $l$ items of user's interaction history: $s_u=(i_1, i_2, ..., i_l)$.

$$
\bold{p}_u = (\bold{Q}^\top \bold{C}_u\bold{Q}+\lambda \bold{I})^{-1}\bold{Q}^\top \bold{C}_u (a_u-h_u(\bold{Q})) 
$$

$$
\frac{\partial L}{\partial \bold{Q}} = \lambda \bold{Q} + \sum_{u = 1}^M\bold{F}(u)
$$

$$
\bold{F}(u) = \bold{D}_u\bold{e}_N \bold{p}_u^\top + (\bold{D}_u \bold{S}_u + \bold{S}_u^\top\bold{D}_u)\bold{Q}
$$

$$
\bold{D}_u = \text{diag}(\bold{C}_u(\bold{r}(\bold{Q}, \bold{p}_u) - \bold{a}_u))
$$

In [ ]:
class SeqMF:
    def __init__(self, model_config) -> None:
        self.model_config = model_config
        self.last_n = model_config['last_n']
        self.gamma = model_config['gamma']
        self.beta = model_config['beta']
        self.lambd = model_config['lambda']
        self.n_items = model_config['n_items']
        self.dim = model_config['dim']
        sigma = model_config.get('sigma', 0.01)
        self.P = sigma * np.random.randn(model_config['n_users'], model_config['dim'])
        self.Q = sigma * np.random.randn(model_config['n_items'], model_config['dim'])
        self.S = []
        self.C = []
        
    
    def h(self, user, history):
        last_n_items = ...
        return ...
    
    def precompute(self, data, data_description):
        '''
        Builds C_u and S_u matrices for each user.
        C_u is weighted frequency of user's item interaction
        S_u is the item transition matrix for the user
        '''
        # for u in tqdm(range(len(histories)), desc='Build C&S'):
        histories = generate_histories(data, data_description)
        interactions = generate_interactions_matrix(data, data_description)
        
        for u in range(len(histories)):
            weights = interactions[u, :].A.squeeze() ** self.gamma
            C_u = diags(weights / weights.sum())
            self.C.append(C_u)

            rules = {}
            denominator = np.zeros((self.n_items))
            for i in range(1, len(histories[u])):
                if (histories[u][i - 1], histories[u][i]) not in rules:
                    rules[(histories[u][i - 1], histories[u][i])] = 0
                rules[(histories[u][i - 1], histories[u][i])] += 1
                denominator[histories[u][i - 1]] += 1
                
            # create a sparse matrix
            items, values = zip(*rules.items())
            i1, i2 = zip(*items)
            matrix_shape = (data_description['n_items'], data_description['n_items'])
            S_u = coo_matrix((values, (list(i1), list(i2))), shape=matrix_shape).tocsr()
                
            S_u = S_u @ diags(np.divide(1.0, denominator, where=(denominator!=0), out=denominator))

            self.S.append(S_u)

    def fit(self, data, data_description, n_epochs):
        # !!! hybrid optimization scheme !!!
        # user factors are updated using ALS
        # item factors are updated using GD

        if len(self.S) == 0:
            self.precompute(data, data_description)
            
        interactions = generate_interactions_matrix(data, data_description)
        histories = generate_histories(data, data_description)
                
        for epoch in range(n_epochs):
            # updating user factors through ALS scheme
            # for u in tqdm(range(len(histories)), desc='update user'):
            for u in tqdm(range(len(histories)), desc='Updating user factors'):
                self.P[u, :] = ...
                
            # updating item factors via GD
            grad = self.lambd * self.Q
            u_ids = np.random.permutation(len(histories))
            for u in tqdm(u_ids, desc='Updating item factors'):
                D_u = ...
                grad += ...
            self.Q -= self.beta * grad
    
    def folding_in(self, data, data_description, n_epochs=None):
        C_warm = []
        warm_interactions = generate_interactions_matrix(data, data_description, rebase_users=True)
        warm_histories = generate_histories(data, data_description)
        n_warm_users = warm_interactions.shape[0]
        
        
        for u in range(n_warm_users):
            weights = ...
            C_u = ...
            C_warm.append(C_u)
            
        
        P_warm = np.zeros((n_warm_users, self.dim))
        
        for u in range(n_warm_users):
            P_warm[u, :] = ...

        scores = ...
        return scores


In [ ]:
seqmf_config = {
    'n_items':data_description['n_items'],
    'n_users':data_description['n_users'],
    'dim':16,
    'last_n':3,
    'beta':0.01,
    'lambda':0.5,
    'gamma':0.5,
}


seqmf_model = SeqMF(seqmf_config)

factor_norms = {
    'P':[],
    'Q':[],
}

In [ ]:
n_epochs = 10
for epoch in tqdm(range(n_epochs)):
    seqmf_model.fit(training, data_description, 1)
    
    factor_norms['P'].append(np.linalg.norm(seqmf_model.P, 'fro'))
    factor_norms['Q'].append(np.linalg.norm(seqmf_model.Q, 'fro'))
    
    scores_val = seqmf_model.folding_in(testset_valid, data_description)
    downvote_seen_items(scores_val, testset_valid, data_description)
    recs = topn_recommendations(scores_val)
    print(f'epoch {epoch + 1}: {model_evaluate(recs, holdout_valid, data_description)}')


In [ ]:
scores = seqmf_model.folding_in(testset, data_description)
downvote_seen_items(scores, testset, data_description)
recs = topn_recommendations(scores)
print(model_evaluate(recs, holdout, data_description))

In [ ]:
import matplotlib.pyplot as plt

for factor in factor_norms.keys():
    plt.plot(factor_norms[factor], label=factor)
    
plt.ylabel('Frobenius norm')
plt.xlabel('Epoch')
plt.legend()
plt.show()